In [ ]:
!f2py --f90exec=mpif90 -I. -c -m ocean ocean.f90

In [ ]:
import time
import numpy as np

import wave as ocean

%matplotlib inline

# Set-up bathymetry grid

In [ ]:
file1='../data/topoGBR1000.csv'
file2='../data/gbr_south.csv'
file3='../data/OTR.csv'

waveparams = ocean.wave(file1,wavebase=20,resfac=3,dia=0.0001)

# Wave climate initialisation

Define wave parameters and sea level

In [ ]:
# Sea level position
slvl = 0.
# Significant wave height [L]
H0 = 2

# Define wave source at boundary
src = np.zeros(waveparams.sregZ.shape)
#src = np.zeros(waveparams.regZ.shape)
src.fill(-2)
#src[:,-1] = 0.
src[-1,0] = 0.
#src[100,0] = 0.

Define wave region computational grid

In [ ]:
t0 = time.clock()
waveparams.findland(slvl)
print 'Wave region computation took (s):',time.clock()-t0

# Waves characteristics

These initial waves are then transformed from deep to shallow water assuming shore-parallel depth contours. The orientation of wave fronts is determine by wave celerity and refraction due to depth variations and travel time in the domain is calculated from Huygen's principle (using an order $\sqrt{5}$ approximation).

## Wave shoaling

The coefficient n relates wave velocity to the velocity of a group of waves (Cg=Cn) describing the evolution of wave shape with shoaling.

Assuming no refraction or loss of energy due to bottom friction, wave power P is conserved from deep to shallow water. The associated change in wave height can be described using a shoaling coefficient KS found by equating the power of deep water waves P0 to that of shallow waves P:
 $$Ks = \sqrt{\frac{1}{2n}}$$
Shoaling results in a slight reduction in wave height as a wave enters intermediate water depths, but approaching shallow water, wave height increases.

In [ ]:
t0 = time.clock()
waveparams.cmptwaves(src, h0=H0, sigma=1.)
print 'Wave parameters computation took (s): ',time.clock()-t0

## Visualisation of wave characteristics

In [ ]:
size = (20,40)
i1 = 0 
i2 = -1
j1 = 0
j2 = -1

# waveparams.plotData(data='bathy', figsize=size, vmin=0, vmax=0, 
#                  fontsize=10, imin=i1, imax=i2, jmin=j1, jmax=j2)

# waveparams.plotData(data='travel', figsize=size, tstep=800, vmin=0, vmax=0, 
#                  fontsize=10, imin=i1, imax=i2, jmin=j1, jmax=j2)

# waveparams.plotData(data='wcelerity', figsize=size, vmin=0, vmax=15, 
#                  fontsize=10, stream=3, imin=i1, imax=i2, jmin=j1, jmax=j2)

# waveparams.plotData(data='ubot', figsize=size, vmin=0, vmax=2, 
#                  fontsize=10, imin=i1, imax=i2, jmin=j1, jmax=j2)

# waveparams.plotData(data='shear', figsize=size, vmin=-0.5, vmax=0.5, 
#                   fontsize=10, imin=i1, imax=i2, jmin=j1, jmax=j2)

# Sediment transport

In [ ]:
t0 = time.clock()
waveparams.cmptsed(sigma=1.,tsteps=500,dsteps=1000)
print 'Sediment erosion/deposition computation took (s): ',time.clock()-t0

## Visualisation of sediment transport characteristics

In [ ]:
size = (10,20)
i1 = 0 
i2 = -1 
j1 = 0
j2 = -1 

waveparams.plotData(data='fbathy', figsize=size, vmin=0, vmax=0, 
                 fontsize=10, stream=0, imin=i1, imax=i2, jmin=j1, jmax=j2)

waveparams.plotData(data='erodep', figsize=size, vmin=-0.5, vmax=0.5, 
                 fontsize=10, stream=0, imin=i1, imax=i2, jmin=j1, jmax=j2)

In [ ]:
waveparams.outputCSV(filename='erodep.csv', seddata=1)